# Custom deploy Model End-point

## 1. Project Variables

In [1]:
import os, sys
import json

ENDPOINT_ID = os.getenv("ENDPOINT_CONFIG_ID") # Identificador unico
ENDPOINT_COMPANY = os.getenv("ENDPOINT_CONFIG_COMPANY", "Cinemas") # Nombre de la compañia
ENDPOINT_COMPANY_ABBR = os.getenv("ENDPOINT_CONFIG_COMPANY_ABBR", "Ci") # Abreviatura de la compañia

ENDPOINT_DISPLAY_NAME = os.getenv("ENDPOINT_DISPLAY_NAME", "ENDPOINTSent")
ENDPOINT_DESCRIPTION = os.getenv("ENDPOINT_DESCRIPTION", "Vertex Module Proyect")

ENDPOINT_SERVICE_ACCOUNT = os.getenv("ENDPOINT_SERVICE_ACCOUNT") # Cuenta de Servicio que se asignara al ENDPOINT
ENDPOINT_PROJECT_ID = os.getenv("ENDPOINT_PROJECT_ID", "ProjectVertexAI") # Proyecto GCP donde se ejecuta el ENDPOINT
ENDPOINT_REGION = os.getenv("ENDPOINT_REGION", "us-central1") # Region
ENDPOINT_BUCKET = os.getenv("ENDPOINT_BUCKET", "proyect-vertex-ai") # Bucket donde se guardaran artefactos del despliegue

ENDPOINT_PATH_ROOT = os.getenv("ENDPOINT_PATH_ROOT", "endpoint")
ENDPOINT_MODEL_PATH_ROOT = os.getenv("ENDPOINT_MODEL_PATH_ROOT", "endpoint/model")

## 2. Endpoint Variables

In [2]:
PROJECT_ID          = ENDPOINT_PROJECT_ID
REGION              = ENDPOINT_REGION
#BUCKET_NAME         = "gs://" + ENDPOINT_BUCKET 
BUCKET_NAME         = ENDPOINT_BUCKET
MODEL_NAME          = 'Sentiment_Analysis.h5'
SAVED_MODEL_DIR    = 'saved_model'
SERVICE_ACCOUNT     = ENDPOINT_SERVICE_ACCOUNT

In [3]:
#ENDPOINT_ROOT = os.path.join(BUCKET_NAME, ENDPOINT_PATH_ROOT)
ENDPOINT_MODEL_ROOT = os.path.join(ENDPOINT_MODEL_PATH_ROOT, MODEL_NAME)
ENDPOINT_SAVED_MODEL_ROOT = os.path.join(ENDPOINT_MODEL_PATH_ROOT, SAVED_MODEL_DIR)
#print(ENDPOINT_ROOT)
print(ENDPOINT_SAVED_MODEL_ROOT)

endpoint/model/saved_model


## 3. Instalar TensorFlow

In [4]:
!pip install tensorflow

In [4]:
import tensorflow as tf

2023-10-10 04:30:37.696781: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-10 04:30:37.696861: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-10 04:30:37.702485: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-10 04:30:38.200909: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 4. Crear y entrenar el modelo

In [6]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Parámetros del modelo
max_features = 5000  # Número máximo de palabras a considerar como características
max_len = 200  # Longitud máxima de la secuencia de texto
embedding_dim = 128  # Dimensión de los vectores de palabra de la capa Embedding

# Cargar el conjunto de datos IMDB
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# Preprocesamiento de datos
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

# Crear el modelo
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=embedding_dim, input_length=max_len))
model.add(LSTM(units=256))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(x_train, y_train, batch_size=64, epochs=3, validation_split=0.2)

# Evaluar el modelo
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Pérdida del modelo: {loss:.4f}")
print(f"Precisión del modelo: {accuracy*100:.2f}%")

Epoch 1/3
313/313 [==============================] - 279s 883ms/step - loss: 0.4775 - accuracy: 0.7646 - val_loss: 0.3464 - val_accuracy: 0.8548
Epoch 2/3
313/313 [==============================] - 289s 923ms/step - loss: 0.3007 - accuracy: 0.8767 - val_loss: 0.3604 - val_accuracy: 0.8420
Epoch 3/3
782/782 [==============================] - 163s 208ms/step - loss: 0.3679 - accuracy: 0.8563
Pérdida del modelo: 0.3679
Precisión del modelo: 85.63%


In [7]:
model.summary()
model.save('Sentiment_Analysis.h5')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          640000    
                                                                 
 lstm (LSTM)                 (None, 256)               394240    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 1034497 (3.95 MB)
Trainable params: 1034497 (3.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## 5. Crear Endpoint

### 5.1 Exportar a saved_model (artefacto)

In [9]:
# Exportar el modelo a saved_model
tf.saved_model.save(model, './saved_model')

INFO:tensorflow:Assets written to: ./saved_model/assets


INFO:tensorflow:Assets written to: ./saved_model/assets


### 5.2  Subir el model.h5 y saved_model a GCS

In [5]:
from google.cloud import storage

def upload_to_gcs(bucket_name, local_file_path, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(local_file_path)

# Especifica el nombre del bucket de GCS y los nombres de archivo
bucket_name = BUCKET_NAME#ENDPOINT_MODEL_ROOT#'nombre_del_bucket'
local_file_path = MODEL_NAME
destination_blob_name = ENDPOINT_MODEL_ROOT

# Sube el archivo al bucket de GCS
upload_to_gcs(bucket_name, local_file_path, destination_blob_name)

In [6]:
import os
from google.cloud import storage

def upload_directory_to_gcs(bucket_name, local_directory, destination_directory):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    for root, dirs, files in os.walk(local_directory):
        for file in files:
            local_path = os.path.join(root, file)
            destination_path = os.path.join(destination_directory, os.path.relpath(local_path, local_directory))
            blob = bucket.blob(destination_path)
            blob.upload_from_filename(local_path)
            print(f'Subido: {local_path} -> gs://{bucket_name}/{destination_path}')

# Especifica el nombre del bucket de GCS y los directorios local y de destino
bucket_name = BUCKET_NAME
local_directory = SAVED_MODEL_DIR
destination_directory = ENDPOINT_SAVED_MODEL_ROOT

# Sube el directorio al bucket de GCS
upload_directory_to_gcs(bucket_name, local_directory, destination_directory)


Subido: saved_model/fingerprint.pb -> gs://proyect-vertex-ai/endpoint/model/saved_model/fingerprint.pb
Subido: saved_model/saved_model.pb -> gs://proyect-vertex-ai/endpoint/model/saved_model/saved_model.pb
Subido: saved_model/variables/variables.index -> gs://proyect-vertex-ai/endpoint/model/saved_model/variables/variables.index
Subido: saved_model/variables/variables.data-00000-of-00001 -> gs://proyect-vertex-ai/endpoint/model/saved_model/variables/variables.data-00000-of-00001


### 5.3 Subir modelo a registry

In [5]:
!pip install -U google-cloud-aiplatform "shapely<2"

In [7]:
from google.cloud import aiplatform
from typing import Optional

def upload_model_sample(
    project: str,
    location: str,
    display_name: str,
    serving_container_image_uri: str,
    artifact_uri: Optional[str] = None
):

    aiplatform.init(project=project, location=location)
    
    model = aiplatform.Model.upload(
        display_name=display_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=serving_container_image_uri,
    )
    
    model.wait()

    print(model.display_name)
    print(model.resource_name)
    return model


In [8]:
#BUCKET_NAME         = "gs://" + ENDPOINT_BUCKET 
ARTIFACT_URI           = "gs://" + os.path.join(ENDPOINT_BUCKET, destination_directory)
print(ARTIFACT_URI)

gs://proyect-vertex-ai/endpoint/model/saved_model


In [9]:
#upload_model_sample(project= PROJECT_ID, location = REGION, display_name= 'keras_registry', serving_container_image_uri = destination_directory, artifact_uri = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-11:latest')
#'gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-5:latest'
keras_model_registry = upload_model_sample(project= "projectvertexai-401304", \
                    location = REGION, \
                    display_name= 'proyect_vertex', \
                    serving_container_image_uri = 'gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-5:latest', \
                    artifact_uri = ARTIFACT_URI)

Creating Model
Create Model backing LRO: projects/297594721003/locations/us-central1/models/4255433255911686144/operations/5884005346120302592
Model created. Resource name: projects/297594721003/locations/us-central1/models/4255433255911686144@1
To use this Model in another session:
model = aiplatform.Model('projects/297594721003/locations/us-central1/models/4255433255911686144@1')
proyect_vertex
projects/297594721003/locations/us-central1/models/4255433255911686144


In [10]:
name_model_registry = keras_model_registry.name

In [11]:
print(name_model_registry)

4255433255911686144


### 5.4 Creación del Endpoint

In [12]:
def create_endpoint_sample(
    project: str,
    display_name: str,
    location: str,
):
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint.create(
        display_name=display_name,
        project=project,
        location=location,
    )

    print(endpoint.display_name)
    print(endpoint.resource_name)
    return endpoint

In [13]:
endpoint = create_endpoint_sample(project= "projectvertexai-401304", \
                                    display_name= 'Sentiment_Analysis_endpoint', \
                                    location= REGION)

Creating Endpoint
Create Endpoint backing LRO: projects/297594721003/locations/us-central1/endpoints/1853391775360614400/operations/7370193223152566272
Endpoint created. Resource name: projects/297594721003/locations/us-central1/endpoints/1853391775360614400
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/297594721003/locations/us-central1/endpoints/1853391775360614400')
Sentiment_Analysis_endpoint
projects/297594721003/locations/us-central1/endpoints/1853391775360614400


In [14]:
ENDPOINT_NAME = 'Sentiment_Analysis_endpoint'
!gcloud ai endpoints list \
  --region=$REGION\
  --filter=display_name=$ENDPOINT_NAME

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
ENDPOINT_ID          DISPLAY_NAME
1853391775360614400  Sentiment_Analysis_endpoint
1941211968094339072  Sentiment_Analysis_endpoint


### 5.5 Deploy Model to Endpoint

In [22]:
!pip install shap

In [15]:
from typing import Dict, Sequence, Tuple
#import shap.explain

def deploy_model_with_dedicated_resources_sample(
    project,
    location,
    model_name: str,
    machine_type: str,
    endpoint: Optional[aiplatform.Endpoint] = None,
    deployed_model_display_name: Optional[str] = None,
    traffic_percentage: Optional[int] = 0,
    traffic_split: Optional[Dict[str, int]] = None,
    min_replica_count: int = 1,
    max_replica_count: int = 1,
    accelerator_type: Optional[str] = None,
    accelerator_count: Optional[int] = None,
    #explanation_metadata: Optional[explain.ExplanationMetadata] = None,
    #explanation_parameters: Optional[explain.ExplanationParameters] = None,
    metadata: Optional[Sequence[Tuple[str, str]]] = (),
    sync: bool = True,
):
    """
    model_name: A fully-qualified model resource name or model ID.
          Example: "projects/123/locations/us-central1/models/456" or
          "456" when project and location are initialized or passed.
    """

    aiplatform.init(project=project, location=location)
    print('model name: ', model_name)
    model = aiplatform.Model(model_name=model_name)
    #model = aiplatform.Model(display_name=model_name)
    
    # The explanation_metadata and explanation_parameters should only be
    # provided for a custom trained model and not an AutoML model.
    model.deploy(
        endpoint=endpoint,
        deployed_model_display_name=deployed_model_display_name,
        traffic_percentage=traffic_percentage,
        traffic_split=traffic_split,
        machine_type=machine_type,
        min_replica_count=min_replica_count,
        max_replica_count=max_replica_count,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        #explanation_metadata=explanation_metadata,
        #explanation_parameters=explanation_parameters,
        metadata=metadata,
        sync=sync,
    )
    
    model.wait()

    print(model.display_name)
    print(model.resource_name)
    return model

In [16]:
deploy_model_with_dedicated_resources_sample(
    project="projectvertexai-401304", \
    location=REGION, \
    model_name= name_model_registry, \
    machine_type='n1-standard-4', \
    endpoint= endpoint, \
    deployed_model_display_name='Sentiment_Analysis_model_endpoint', \
    #traffic_percentage: Optional[int] = 0,
    #traffic_split: Optional[Dict[str, int]] = None,
    min_replica_count= 1, \
    max_replica_count= 2
    )

model name:  4255433255911686144
Deploying model to Endpoint : projects/297594721003/locations/us-central1/endpoints/1853391775360614400
Deploy Endpoint model backing LRO: projects/297594721003/locations/us-central1/endpoints/1853391775360614400/operations/7993941771543379968
Endpoint model deployed. Resource name: projects/297594721003/locations/us-central1/endpoints/1853391775360614400
proyect_vertex
projects/297594721003/locations/us-central1/models/4255433255911686144


resource name: projects/297594721003/locations/us-central1/models/4255433255911686144

### 5.6 Obtener una predicción

In [60]:
def endpoint_predict_sample(
    project: str, location: str, instances: str, endpoint: str
):
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint(endpoint)

    prediction = endpoint.predict(instances=instances)
    print(prediction)
    return prediction

In [61]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [47]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Parámetros del modelo
max_features = 5000  # Número máximo de palabras a considerar como características
max_len = 200  # Longitud máxima de la secuencia de texto
embedding_dim = 128  # Dimensión de los vectores de palabra de la capa Embedding

# Cargar el conjunto de datos IMDB
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# Preprocesamiento de datos
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

In [84]:
input_data = "Esta película no estuvo mal, pero no me gustó"
print("datos de entrada: \n", input_data)
word_index = imdb.get_word_index()
max_len = 200
max_features = 5000
# Preprocesar la revisión de muestra y convertirla en una secuencia numérica
input_data = [word_index[word] if word in word_index and word_index[word] < max_features else 0 for word in input_data.split()]
input_data = pad_sequences([input_data], maxlen=max_len)

datos de entrada: 
 Esta película no estuvo mal, pero no me gustó


In [85]:
input_data = input_data.tolist()

In [86]:
print(input_data)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 54, 0, 0, 0, 54, 69, 0]]


In [87]:
response = endpoint_predict_sample(
    project= "projectvertexai-401304" ,\
    location= REGION ,\
    instances= input_data ,\
    endpoint= endpoint.name 
)

Prediction(predictions=[[0.415145695]], deployed_model_id='7237946507184308224', model_version_id='1', model_resource_name='projects/297594721003/locations/us-central1/models/4255433255911686144', explanations=None)
